In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tensorflow.keras.applications import ResNet50

In [3]:
import numpy as np
from sklearn import metrics

In [16]:
from tensorflow.keras import Model, Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import LearningRateScheduler

In [5]:
import json
import os

from pprint import pprint

In [6]:
from data.coco.coco import CocoGenerator

Using TensorFlow backend.


In [15]:
from model.callbacks.metric_callbacks import MAPCallback
from model.callbacks.save_callback import SaveModel
from model.callbacks.scheduler import lr_scheduler

In [8]:
from model.losses import BCE, PartialBCE

In [17]:
from config import config_utils
from model.utils.config import cfg
from experiments.launch import parse_options_file

In [9]:
prop = 100

In [14]:
parse_options_file('/home/caleml/partial-labels/config/pv_baseline50_sgd_448lrs.yaml')


Loaded config

{'archi': {'classifier': 'resnet50', 'loss': 'bce', 'name': 'baseline'},
 'batch_size': 32,
 'callbacks': {'lr_factor': 0.1, 'lr_trigger': [10]},
 'dataset': {'name': 'pascalvoc',
             'path': '/home/caleml/datasets/pascalvoc/VOCdevkit/VOC2007/',
             'test': 'test',
             'train': 'trainval'},
 'image': {'img_size': 448},
 'training': {'n_epochs': 20, 'optimizer': 'sgd', 'start_lr': 0.01}}



{'archi': {'classifier': 'resnet50', 'loss': 'bce', 'name': 'baseline'},
 'batch_size': 32,
 'callbacks': {'lr_factor': 0.1, 'lr_trigger': [10]},
 'dataset': {'name': 'pascalvoc',
  'path': '/home/caleml/datasets/pascalvoc/VOCdevkit/VOC2007/',
  'test': 'test',
  'train': 'trainval'},
 'image': {'img_size': 448},
 'training': {'n_epochs': 20, 'optimizer': 'sgd', 'start_lr': 0.01}}

# datasets

In [ ]:
# train
train_dataset = CocoGenerator('train', '/share/DEEPLEARNING/datasets/mscoco', prop=prop)

In [ ]:
generator_train = train_dataset.flow(batch_size=2)
X_train, Y_train = next(generator_train)

In [ ]:
print(X_train.shape)
print(Y_train.shape)

In [ ]:
# validation
val_dataset = CocoGenerator('val', '/share/DEEPLEARNING/datasets/mscoco')

In [ ]:
print(len(val_dataset.image_ids_in_subset))

In [ ]:
batch_size = len(val_dataset.image_ids_in_subset)
generator_val = val_dataset.flow(batch_size=batch_size)
X_test, Y_test = next(generator_val)
print(X_test.shape, Y_test.shape)

# model

In [ ]:
# dropping the Dense(1000)
model = ResNet50(include_top=True, weights='imagenet')
x = model.layers[-2].output
x = Dense(train_dataset.nb_classes, activation='sigmoid', name='predictions')(x)
model = Model(inputs=model.input, outputs=x)

In [ ]:
model.summary()

In [ ]:
lr = 0.1
loss = BCE()
# loss = PartialBCE(prop / 100)
model.compile(loss=loss, optimizer=SGD(lr=lr), metrics=['binary_accuracy'])

# train

In [ ]:
# callbacks
exp_folder = '/home/caleml/partial_experiments/exp_20190723_0050_TESTNB'
os.makedirs(exp_folder, exist_ok=True)

cb_list = list()
cb_list.append(SaveModel(exp_folder, 100))

map_cb = MAPCallback(X_test, Y_test, exp_folder, prop=prop)
cb_list.append(map_cb)

cb_list.append(LearningRateScheduler(lr_scheduler))

In [ ]:
batch_size=32
nb_epochs=20

steps_per_epoch_train = int(len(train_dataset.id_to_label) / batch_size) + 1
model.fit_generator(train_dataset.flow(batch_size=batch_size),
                    steps_per_epoch=steps_per_epoch_train,
                    epochs=nb_epochs,
                    callbacks=cb_list,
                    verbose=1)